## Doing Augmentation

https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

## Visualize The label per image

In [ ]:
loc_img = "dummy/test/images/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.jpg"
loc_annot = "dummy/test/labels/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.txt"
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread(loc_img)
with open(loc_annot,'r') as f:
    value = f.readline().strip()
    value_list = value.split()
value_list = [float(val)for val in value_list]

coords = value_list[1:]

augmented = augmentor(image=img,bboxes=[coords],class_labels=['face'])

# Assuming augmented['image'] is the augmented image you want to draw on
augmented_img =augmented['image']

# Assuming augmented['bboxes'][0] contains the augmented bounding box coordinates
augmented_bbox = augmented['bboxes'][0]

# Original image size
original_width = 640
original_height = 480

# Augmented image size
augmented_width = 450
augmented_height = 450

# Calculate the scale factors for width and height
width_scale = augmented_width / original_width
height_scale = augmented_height / original_height

# Scale the augmented bounding box coordinates to match the augmented image size
x_center = augmented_bbox[0] * original_width  # x-coordinate of the box center
y_center = augmented_bbox[1] * original_height  # y-coordinate of the box center
box_width = augmented_bbox[2] * original_width  # width of the box
box_height = augmented_bbox[3] * original_height  # height of the box

# Calculate the top-left and bottom-right coordinates of the box
x_min = int((x_center - (box_width / 2)) * width_scale)
y_min = int((y_center - (box_height / 2)) * height_scale)
x_max = int((x_center + (box_width / 2)) * width_scale)
y_max = int((y_center + (box_height / 2)) * height_scale)

new_coords = [x_min,y_min,x_max,y_max]

# Draw the rectangle on the augmented image
color = (255, 0, 0)  # Blue color
thickness = 2  # Thickness of the rectangle border
cv2.rectangle(augmented_img, (x_min, y_min), (x_max, y_max), color, thickness)

# Display the augmented image with the rectangle
plt.imshow(augmented_img)
plt.show()

## making suitable (Finalized Version) Augmentation Pure code

In [1]:
import cv2
import albumentations as alb
import os

In [2]:
W = 1280
H = 720

In [3]:
augmentor_without_boudingbox = alb.Compose([
    # alb.RandomCrop(width=W, height=H), 
            alb.HorizontalFlip(p=0.5), 
            alb.RandomBrightnessContrast(p=0.5),
            alb.Rotate(limit=30, p=0.5),
            alb.Blur(p=0.2),
            alb.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.2),
            alb.RandomGamma(p=0.5), 
            alb.RGBShift(p=0.5), 
            alb.VerticalFlip(p=0.5)])
augmentor = alb.Compose([
    # alb.RandomCrop(width=W, height=H), 
                        alb.HorizontalFlip(p=0.5), 
                        alb.RandomBrightnessContrast(p=0.5),
                        alb.Rotate(limit=30, p=0.5),
                        alb.Blur(p=0.2),
                        alb.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.2),
                        alb.RandomGamma(p=0.5), 
                        alb.RGBShift(p=0.5), 
                        alb.VerticalFlip(p=0.5)], 
                    bbox_params=alb.BboxParams(format='yolo', 
                                                label_fields=['class_labels']))

In [4]:
import os
import cv2,shutil

In [ ]:
for i in os.listdir("Test/images/"):
    print(i)
    img_dir = "Test/images/"+i
    label_dir = "Test/labels/"+i.split(".")[0] + ".txt"
    image = i # image name 
    img = cv2.imread(img_dir)
    coords_list = []
    class_value = []
    label_path = os.path.join(label_dir)
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            content = f.readlines()
        for line in content:
            label = [float(value) for value in line.strip().split(" ")]
            class_value.append(int(label[0]))
            coords = label[1:5]
            coords_list.append(coords)
    for x in range(12):
        if coords_list:
            augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
        else:
            augmented = augmentor_without_boudingbox(image=img)
        partition = "Traintesting" # don't change this 
        directory_path = os.path.join('dummy', partition, 'images')
        os.makedirs(directory_path, exist_ok=True)
        cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
        annotation = []
        if os.path.exists(label_path):
            for i in range(len(coords_list)):
                if i < len(augmented['bboxes']):
                    annotation.append(class_value[i])
                    annotation.extend(augmented['bboxes'][i])
                else:
                    annotation.append(0)
                    annotation.extend([0, 0, 0, 0])
        else:
            annotation.append(0)
            annotation.append(0)
            annotation.append(0)
            annotation.append(0)
            annotation.append(0)

        directory_path_labels = os.path.join('dummy', partition, 'labels')
        os.makedirs(directory_path_labels, exist_ok=True)
        with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),'w') as f:
            f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))

In [5]:
for partition in ['Splitted/Train',"Splitted/Valid/","Splitted/Test/"]: 
    print(f"doing -> {partition}")
    for image in os.listdir(os.path.join(partition, 'images')):
        img = cv2.imread(os.path.join(partition, 'images', image))
        coords_list = []
        class_value = []
        label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.readlines()
            for line in content:
                label = [float(value) for value in line.strip().split(" ")]
                class_value.append(int(label[0]))
                coords = label[1:5]
                coords_list.append(coords)
        for x in range(3):
            try:
                if coords_list:
                    augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
                else:
                    augmented = augmentor_without_boudingbox(image=img)
                directory_path = os.path.join('dummy', partition, 'images')
                os.makedirs(directory_path, exist_ok=True)
                cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
                annotation = []
                if os.path.exists(label_path):
                    for i in range(len(coords_list)):
                        if i < len(augmented['bboxes']):
                            annotation.append(class_value[i])
                            annotation.extend(augmented['bboxes'][i])
                        else:
                            annotation.append(0)
                            annotation.extend([0, 0, 0, 0])
                else:
                    annotation.append(0)
                    annotation.append(0)
                    annotation.append(0)
                    annotation.append(0)
                    annotation.append(0)
        
                directory_path_labels = os.path.join('dummy', partition, 'labels')
                os.makedirs(directory_path_labels, exist_ok=True)
                with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),'w') as f:
                    f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))
            except:
                # print("Skipping!")
                pass
                

doing -> Splitted/Train
doing -> Splitted/Valid/
doing -> Splitted/Test/


## Visualizing Images code

In [62]:
import cv2
import random
import os

org_image = "Garden4.jpg"
org_label = "Garden4.txt"
split = "Valid"
image_chosen = random.sample(os.listdir(f"New_Augmented_Data/{split}/images/"),1)[0].replace(".jpg","")
# "New_Augmented_Image\train\images\a4_16.0.jpg"
image_path = f"New_Augmented_Data//{split}//images//{image_chosen}.jpg"
label_path = f"New_Augmented_Data//{split}//labels//{image_chosen}.txt"

image = cv2.imread(image_path)
# Resize image for display
scale_percent = 50  # Adjust the value as needed
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
image = cv2.resize(image, (width, height))

with open(label_path, 'r') as f:
    lines = f.readlines()

color_mapping = {
    0: (255, 0, 0),  # Class 0 will be displayed in blue (BGR format)
    1: (0, 255, 233),  # Class 1 will be displayed in green
    2: (0, 0, 255)   # Class 2 will be displayed in red
}

for line in lines:
    label = line.strip().split(" ")
    class_id = int(label[0])
    x_center = float(label[1]) * image.shape[1]
    y_center = float(label[2]) * image.shape[0]
    width = float(label[3]) * image.shape[1]
    height = float(label[4]) * image.shape[0]

    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    # Draw bounding box rectangle
    color = color_mapping.get(class_id, (0, 0, 0))
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

    # Display class label
    class_label = f"{class_id}"
    cv2.putText(image, class_label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Display the image with bounding box annotations
cv2.imshow("Image with Bounding Boxes", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Rename Files do our desired names

In [ ]:
import os
def Rename_Files(Image_Dir_List,Label_Dir_List,New_Name):
    folder_images_path = Image_Dir_List
    folder_label_path = Label_Dir_List
    Give_Me_Name = New_Name

    for dir in folder_images_path:
        file_list = os.listdir(dir)

        for i, filename in enumerate(file_list):
            new_filename = f"{Give_Me_Name}{i+1}.jpg"
            old_filepath = os.path.join(dir, filename)
            new_filepath = os.path.join(dir, new_filename)
            os.rename(old_filepath, new_filepath)
            
            print(f"Renamed {filename} to {new_filename}")
    for dir in folder_label_path:
        file_list = os.listdir(dir)

        for i,filename in enumerate(file_list):
            new_filename = f"{Give_Me_Name}{i+1}.txt"
            old_filepath = os.path.join(dir,filename)
            new_filepath = os.path.join(dir,new_filename)
            os.rename(old_filepath,new_filepath)

            print(f"Renamed {filename} to {new_filename}")

## Train Val Test Split

In [17]:
import os
import random
import shutil

def split_dataset(image_dir, label_dir, train_ratio,  valid_ratio):
    # Create directories for train, test, and validation sets
    train_dir = "train"
    valid_dir = "valid"
    test_dir = "test"
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)

    # Get a list of all image file names with the .jpg extension in the image directory
    image_files = [f for f in os.listdir(image_dir)]

    # Shuffle the list of image files randomly
    random.shuffle(image_files)

    # Calculate the number of images for each split based on the ratios
    total_images = len(image_files)
    train_split = int(total_images * train_ratio)
    valid_split = int(total_images * valid_ratio)

    # Move images to the respective split directories
    for i, image_file in enumerate(image_files):
        if i < train_split:
            split_dir = train_dir
        elif i < train_split + valid_split:
            split_dir = valid_dir
        else:
            split_dir = test_dir

        # Copy the image file to the split directory
        image_src = os.path.join(image_dir, image_file)
        image_dst = os.path.join(split_dir, "images", image_file)
        os.makedirs(os.path.dirname(image_dst), exist_ok=True)
        shutil.copy(image_src, image_dst)

        # Get the corresponding label file
        label_prefix = os.path.splitext(image_file)[0]
        label_file = label_prefix + ".txt"

        # Copy the label file to the split directory
        label_src = os.path.join(label_dir, label_file)
        label_dst = os.path.join(split_dir, "labels", label_file)
        os.makedirs(os.path.dirname(label_dst), exist_ok=True)
        shutil.copy(label_src, label_dst)

    print("Dataset split completed successfully.")

# Specify the directories and split ratios
image_directory = "to_do_data/images"
label_directory = "to_do_data/labels"
train_ratio = 0.8
valid_ratio = 0.15

# Call the split_dataset function
split_dataset(image_directory, label_directory, train_ratio, valid_ratio)


Dataset split completed successfully.


## To Create Empty Text File According To The Images

In [3]:
import os

image_files = os.listdir("Test/images/")
text_files = []

for image_file in image_files:
    base_name, _ = os.path.splitext(image_file)
    text_file = base_name + ".txt"
    text_files.append(text_file)
    with open(text_file, 'w') as file:
        pass  # Creates an empty file

print("Empty text files created:", text_files)

Empty text files created: ['b_1_9.txt', 'b_2_9.txt', 'b_3_9.txt', 'b_4_9.txt']


## For Debugging Purposes

In [2]:
from WackyWarper.config import albumentation_custom as alb_c
import os
import cv2,shutil

In [39]:
from WackyWarper import helper_functions
n = "20230614145514_c3.0"
l = "20230614145514_c3.0"
helper_functions.Visualize(f"dummy\Train\images/{n}.jpg",f"dummy/Train/labels/{l}.txt")

### Augmentation Code

In [23]:
def ch(header_folder_name, list_of_directory, images_needed):
    print(">>Augmentation started<<")
    folder_name = header_folder_name
    for partition in list_of_directory:
        for image in os.listdir(os.path.join(partition, 'images')):
            img = cv2.imread(os.path.join(partition, 'images', image))
            coords_list = []
            class_value = []
            label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    content = f.readlines()
                for line in content:
                    label = [float(value) for value in line.strip().split(" ")]
                    class_value.append(int(label[0]))
                    coords = label[1:5]
                    coords_list.append(coords)
             
            for x in range(images_needed):
                try:
                    
                    if coords_list:
                        augmented = alb_c.augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
                    else:
                        augmented = alb_c.augmentor_without_boudingbox(image=img)
                    partitionl = partition.split("\\")[1] 
                    directory_path = os.path.join(folder_name, partitionl, 'images')
                    os.makedirs(directory_path, exist_ok=True)
                    # Copy the original image to the augmented folder
                    original_image_path = os.path.join(partition, 'images', image)
                    new_image_path = os.path.join(folder_name, partitionl, 'images', f'{image.split(".")[0]}.jpg')
                    shutil.copyfile(original_image_path, new_image_path)

                    directory_path_labels = os.path.join(folder_name, partitionl, 'labels')
                    os.makedirs(directory_path_labels, exist_ok=True)

                    # Move the original label file to the augmented label folder
                    if coords_list:
                        original_label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
                        new_label_path = os.path.join(folder_name, partitionl, 'labels', f'{image.split(".")[0]}.txt')
                        shutil.copyfile(original_label_path, new_label_path)
                    else:
                        with open(f'{folder_name}\{partitionl}\labels\{image.split(".")[0]}.txt','w') as f:
                            f.write("0 0 0 0 0")

                    cv2.imwrite(os.path.join(folder_name, partitionl, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
                    
                    annotation = []
                    if os.path.exists(label_path):
                        for i in range(len(coords_list)):
                            if i < len(augmented['bboxes']):
                                annotation.append(class_value[i])
                                annotation.extend(augmented['bboxes'][i])
                            else:
                                annotation.append(0)
                                annotation.extend([0, 0, 0, 0])
                    else:
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                    if annotation == []:
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)

                    with open(os.path.join(folder_name, partitionl, 'labels', f'{image.split(".")[0]}.{x}.txt'), 'w') as f:
                        f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))
                except:
                    continue
    print(">>Augmentation Ended<<")

In [24]:
ch("dum", ["Splitted\Train"], 1)

>>Augmentation started<<
>>Augmentation Ended<<


In [36]:
import os
from sklearn.model_selection import train_test_split

valid_ratio = 0.20
test_ratio = 0.05

image_dir = os.listdir("i/")
label_dir = os.listdir("l/")

for img in image_dir:
    label_file = img.split(".")[0] + ".txt"
    label_path = os.path.join("l",label_file)
    if os.path.exists(label_path):
        pass
    else:
        with open(label_path,"w") as f:
            f.write("0 0 0 0 0")

# Split the data into training, validation, and test sets
X_train, X_valid, y_train, y_valid = train_test_split(image_dir, label_dir, test_size=valid_ratio, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=test_ratio, random_state=42)

In [73]:
import shutil

split_dir = "Splitted\Train"

if os.path.exists(split_dir):
    shutil.rmtree(split_dir)

os.makedirs(split_dir, exist_ok=True)

for i in X_train:
    org = os.path.join("i",i)
    dst = os.path.join(split_dir,"Images",i)

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    shutil.copy(org,dst)

for j in y_train:
    org = os.path.join("l",j)
    dst = os.path.join(split_dir,"Labels",j)

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    shutil.copy(org,dst)

split_dir = "Splitted\Valid"

if os.path.exists(split_dir):
    shutil.rmtree(split_dir)

os.makedirs(split_dir, exist_ok=True)

for i in X_valid:
    org = os.path.join("i",i)
    dst = os.path.join(split_dir,"Images",i)

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    shutil.copy(org,dst)

for j in y_valid:
    org = os.path.join("l",j)
    dst = os.path.join(split_dir,"Labels",j)

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    shutil.copy(org,dst)

split_dir = "Splitted\Test"

if os.path.exists(split_dir):
    shutil.rmtree(split_dir)

os.makedirs(split_dir, exist_ok=True)

for i in X_test:
    org = os.path.join("i",i)
    dst = os.path.join(split_dir,"Images",i)

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    shutil.copy(org,dst)

for j in y_test:
    org = os.path.join("l",j)
    dst = os.path.join(split_dir,"Labels",j)

    os.makedirs(os.path.dirname(dst), exist_ok=True)

    shutil.copy(org,dst)